In [ ]:
import sagemaker

In [ ]:
training_input_s3_uri = sagemaker.session.Session().upload_data(path='./data/2-1/training/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='qiita-sagemaker-training/2-1/training')
validation_input_s3_uri = sagemaker.session.Session().upload_data(path='./data/2-1/validation/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='qiita-sagemaker-training/2-1/validation')
test_input_s3_uri = sagemaker.session.Session().upload_data(path='./data/2-1/test/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='qiita-sagemaker-training/2-1/test')

In [ ]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='check.py',
    source_dir = './src/2-1',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={ # ハイパーパラメータはダミーです
        'first-num':5,
        'second-num':2,
        'operator':'m',
        'sagemaker_s3_output':f's3://{sagemaker.session.Session().default_bucket()}/intermediate'
    },
    volume_size=50, # 50GBを指定
)
estimator.fit({
    'training':training_input_s3_uri,
    'validation':validation_input_s3_uri,
    'test': test_input_s3_uri
})

In [ ]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='check.py',
    source_dir = './src/2-1',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={ # ハイパーパラメータはダミーです
        'first-num':5,
        'second-num':2,
        'operator':'m'
    },
    volume_size=50, # 50GBを指定
)
estimator.fit()

In [ ]:
from sagemaker.tensorflow import TensorFlow
git_config = {'repo': 'https://github.com/kazuhitogo/qiita-sagemaker-training'}
estimator = TensorFlow(
    entry_point='check.py',
    source_dir = './src/2-1',
    git_config=git_config,
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={ # ハイパーパラメータはダミーです
        'first-num':5,
        'second-num':2,
        'operator':'m',
        'sagemaker_s3_output':f's3://{sagemaker.session.Session().default_bucket()}/intermediate'
    },
    volume_size=50, # 50GBを指定
)
estimator.fit({
    'training':training_input_s3_uri,
    'validation':validation_input_s3_uri,
    'test': test_input_s3_uri
})

In [ ]:
# 前回のイメージとコードを利用する
image_uri = estimator.latest_training_job.describe()['AlgorithmSpecification']['TrainingImage']
source_tar_gz = estimator.latest_training_job.describe()['HyperParameters']['sagemaker_submit_directory']

In [ ]:
# 前回のイメージとコードを利用する
image_uri = estimator.latest_training_job.describe()['AlgorithmSpecification']['TrainingImage']
source_tar_gz = estimator.latest_training_job.describe()['HyperParameters']['sagemaker_submit_directory']
estimator = sagemaker.estimator.Estimator(
    image_uri=image_uri,
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'first-num':5,
        'second-num':2,
        'operator':'m',
        'sagemaker_s3_output':f's3://{sagemaker.session.Session().default_bucket()}/intermediate',
        'sagemaker_program' : 'check.py',
        'sagemaker_submit_directory' : source_tar_gz
   },
   instance_count=1,
   instance_type='ml.g4dn.xlarge',
)
estimator.fit({
    'training':training_input_s3_uri,
    'validation':validation_input_s3_uri,
    'test': test_input_s3_uri
})

In [ ]:
import boto3
sm_client = boto3.client('sagemaker')

# トレーニングジョブのリストを取得
print(sm_client.list_training_jobs())
# 最後のトレーニングジョブの詳細を取得
print(sm_client.describe_training_job(TrainingJobName=sm_client.list_training_jobs()['TrainingJobSummaries'][0]['TrainingJobName']))

In [ ]:
sm_client.list_training_jobs()['TrainingJobSummaries'][0]['TrainingJobName']